In [13]:
from time import sleep
from ipyleaflet import *
from IPython.display import display
from ipywidgets import FileUpload
import gpxpy

In [14]:
upload = FileUpload(accept='.gpx', multiple=True)
display(upload)

FileUpload(value={}, accept='.gpx', description='Upload', multiple=True)

In [15]:
def get_data(fname):
    points = []
    with open(fname, 'r') as f:
        gpx = gpxpy.parse(f)
        for track in gpx.tracks:
            for segment in track.segments:
                for point in segment.points:
                    points.append((point.latitude, point.longitude,
                                   point.speed, point.time))
    return points
tracks = [get_data(f) for f in upload.value]

In [16]:
t = tracks[0]
topleft = (min(p[0] for p in t), min(p[1] for p in t))
botright = (max(p[0] for p in t), max(p[1] for p in t))
center = (0.5 * (botright[0] + topleft[0]), 0.5 * (botright[1] + topleft[1]))

In [19]:
m = Map(center=center, zoom=13)

image_markers = []
for lat in (topleft[0], botright[0]):
    for long in (topleft[1], botright[1]):
        mark = Marker(location=(lat, long))
        m.add_layer(mark)
        image_markers.append(mark)
 
display(m)

Map(center=[50.48416435, 30.31976985], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…

In [10]:
nb_steps = 10
step = 0.1
start_location = mark.location
end_location = (52.2, 10)

for i in range(nb_steps + 1):
    percentage_animation = i / nb_steps
    
    a = 1 - percentage_animation
    b = percentage_animation
    
    mark.location = (a * start_location[0] + b * end_location[0], a * start_location[1] + b * end_location[1])
    
    sleep(step)
